# Tabular Playground Series - Feb 2021

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## 모듈 불러오기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 데이터 불러오기

In [ ]:
train = pd.read_csv('gdrive/MyDrive/kaggle/Tabular Playground Series - Feb 2021/data/train.csv')
test = pd.read_csv('gdrive/MyDrive/kaggle/Tabular Playground Series - Feb 2021/data/test.csv')
submission = pd.read_csv('gdrive/MyDrive/kaggle/Tabular Playground Series - Feb 2021/data/sample_submission.csv')

In [ ]:
print('Number of Missing values in train dataset:', train.isnull().sum().sum())

Number of Missing values in train dataset: 0


In [ ]:
print('Number of Missing values in test dataset:', test.isnull().sum().sum())

Number of Missing values in test dataset: 0


In [ ]:
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,A,B,A,A,B,D,A,E,C,I,0.923191,0.684968,0.124454,0.217886,0.281421,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
1,2,B,A,A,A,B,B,A,E,A,F,0.437627,0.014213,0.357438,0.846127,0.282354,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
2,3,A,A,A,C,B,D,A,B,C,N,0.732209,0.760122,0.454644,0.812990,0.293756,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
3,4,A,A,A,C,B,D,A,E,G,K,0.705142,0.771678,0.153735,0.732893,0.769785,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
4,6,A,B,A,A,B,B,A,E,C,F,0.486063,0.639349,0.496212,0.354186,0.279105,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974


In [ ]:
test.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,A,B,A,C,B,D,A,E,E,G,0.331261,0.551131,0.486284,0.156058,0.701679,0.595507,0.286912,0.279884,0.202234,0.242654,0.285147,0.264308,0.653654,0.302448
1,5,A,B,A,C,B,D,A,E,C,L,0.402404,0.617546,0.296852,0.187440,0.277480,0.479552,0.397436,0.476742,0.857073,0.516393,0.562065,0.730542,0.318492,0.736251
2,15,A,B,A,C,B,D,A,E,C,F,0.530677,0.618126,0.760865,0.199872,0.279508,0.676395,0.695284,0.253316,0.586934,0.548555,0.836193,0.759788,0.333572,0.273905
3,16,A,A,B,A,B,D,A,E,E,F,0.439653,0.487403,0.715786,0.192808,0.479503,0.759875,0.240049,0.298074,0.442475,0.596746,0.414131,0.255382,0.589080,0.311625
4,17,A,B,A,A,B,B,A,E,E,I,0.389844,0.752760,0.315269,0.191734,0.757845,0.210232,0.329851,0.616663,0.170475,0.263235,0.710961,0.224045,0.285860,0.794931


In [ ]:
target = train['target']
train_id = train['id']
test_id = test['id']

In [ ]:
train_data = train.drop(columns=['id', 'target'])
train_data['type'] = 'train'
test_data = test.drop(columns='id')
test_data['type'] = 'test'

In [ ]:
data = pd.concat([train_data, test_data], ignore_index=True)

In [ ]:
data.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,type
0,A,B,A,A,B,D,A,E,C,I,0.923191,0.684968,0.124454,0.217886,0.281421,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,train
1,B,A,A,A,B,B,A,E,A,F,0.437627,0.014213,0.357438,0.846127,0.282354,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,train
2,A,A,A,C,B,D,A,B,C,N,0.732209,0.760122,0.454644,0.812990,0.293756,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,train
3,A,A,A,C,B,D,A,E,G,K,0.705142,0.771678,0.153735,0.732893,0.769785,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,train
4,A,B,A,A,B,B,A,E,C,F,0.486063,0.639349,0.496212,0.354186,0.279105,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,train


In [ ]:
data[data['type'] == 'train'].describe()

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
count,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000
mean,0.524634,0.506649,0.444115,0.446214,0.455471,0.508337,0.478345,0.455904,0.459321,0.526899,0.504943,0.529938,0.524549,0.503349
std,0.204875,0.235269,0.200089,0.238669,0.200695,0.231612,0.192432,0.204493,0.220642,0.204025,0.201549,0.230860,0.220892,0.225218
min,-0.093505,-0.055105,-0.060274,0.134760,0.189216,-0.087247,0.043953,0.208703,0.004041,0.073040,0.059644,0.064161,-0.005600,0.158121
25%,0.370451,0.352307,0.314121,0.214572,0.279853,0.338747,0.339896,0.278041,0.308655,0.361957,0.338898,0.316662,0.332143,0.291289
50%,0.492208,0.615156,0.457271,0.377823,0.411351,0.441384,0.410090,0.360736,0.425801,0.488867,0.519855,0.558827,0.407365,0.433909
75%,0.654793,0.688150,0.554835,0.719758,0.621808,0.709515,0.604246,0.639388,0.541525,0.752765,0.672809,0.720381,0.732431,0.730870
max,1.052666,0.851746,1.017689,1.006469,0.994050,1.044433,1.093312,1.036541,1.014156,0.972091,1.029773,1.038049,0.961370,0.873579


In [ ]:
data[data['type'] == 'test'].describe()

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.524644,0.505625,0.443344,0.445698,0.454953,0.508761,0.478229,0.455766,0.459130,0.526956,0.505695,0.529122,0.523904,0.503693
std,0.204638,0.235055,0.199677,0.238721,0.200209,0.231695,0.192374,0.204341,0.220842,0.203714,0.202186,0.230578,0.220578,0.225246
min,-0.091359,-0.048022,-0.056000,0.135861,0.190009,-0.057938,0.037044,0.210527,0.000756,0.082659,0.061741,0.070391,0.012898,0.156231
25%,0.370482,0.351918,0.313682,0.214368,0.279851,0.339745,0.340300,0.277950,0.308703,0.361895,0.338309,0.317015,0.332291,0.291208
50%,0.492627,0.614886,0.456888,0.376929,0.411298,0.441504,0.410049,0.360921,0.425561,0.487917,0.522787,0.555936,0.404961,0.437237
75%,0.654624,0.686965,0.553847,0.719421,0.615436,0.710453,0.603509,0.639641,0.541239,0.752074,0.674581,0.719863,0.730765,0.730989
max,1.047206,0.845711,1.021539,1.003975,0.981507,1.041597,1.089515,1.047141,1.014324,0.961880,1.031986,1.039108,0.966387,0.879754


In [ ]:
from sklearn.preprocessing import LabelEncoder
cat_cols = [col for col in data.columns if 'cat' in col]
cont_cols = [col for col in data.columns if 'cont' in col]

for col in cat_cols:
    le = LabelEncoder()
    le.fit(data[col].unique())
    data[col] = le.transform(data[col])

In [ ]:
data.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,type
0,0,1,0,0,1,3,0,4,2,8,0.923191,0.684968,0.124454,0.217886,0.281421,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,train
1,1,0,0,0,1,1,0,4,0,5,0.437627,0.014213,0.357438,0.846127,0.282354,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,train
2,0,0,0,2,1,3,0,1,2,13,0.732209,0.760122,0.454644,0.812990,0.293756,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,train
3,0,0,0,2,1,3,0,4,6,10,0.705142,0.771678,0.153735,0.732893,0.769785,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,train
4,0,1,0,0,1,1,0,4,2,5,0.486063,0.639349,0.496212,0.354186,0.279105,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,train


In [ ]:
def RMSE(y, pred):
    return np.sqrt(np.sum(np.square(y - pred))/len(y))

## 선형 회귀 분석

### Baseline

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data[data['type']=='train'].drop(columns='type'), target, test_size=0.2, random_state=2021)

LR = LinearRegression()
LR.fit(X_train, y_train)
pred = LR.predict(X_val)

print(RMSE(y_val, pred))

0.8675534482786778


### 범주형 변수를 원-핫 인코딩하기

In [109]:
onehot_data = pd.get_dummies(data, columns = cat_cols)

In [111]:
X_train, X_val, y_train, y_val = train_test_split(onehot_data[onehot_data['type']=='train'].drop(columns='type'), target, test_size=0.2, random_state=2021)

In [112]:
LR1 = LinearRegression()
LR1.fit(X_train, y_train)
pred1 = LR1.predict(X_val)

print(RMSE(y_val, pred1))

0.8619102320679294
